In [74]:
import pandas as pd
import numpy as np

import matplotlib.pyplot as plt
from mpl_toolkits.mplot3d import Axes3D
import matplotlib as mpl

import seaborn as sns
from sklearn.neighbors import KNeighborsClassifier
from sklearn.decomposition import PCA


%matplotlib inline

## Measurement Data:
* total 40 feature devided into group of 3
* first 16 feature for left(or right) movement. each group A,B,C,D have 4 feature.
* second 16 feature for right(or left) movement
* last 8 feature is combined feature (A,B,C,D | A,B,C,D)
* total 117 observation in each .txt file

## Human Evaluation
* Represents the minimum limit for testing number of pen
* Total 300 pens are taken and the performance of the pen is evaluated by two employee.
* If either of employee identify the pen with bad quality, we don't count it.
* at the end we find 200 pens are with good quality.

In [75]:
def readData_new(fileName):
    data = pd.read_csv(fileName, skiprows= 10, delimiter= ';', decimal=',',usecols=range(98))
    return data

In [76]:
dataB1 = readData_new('MeasuramentsData/Federtyp B_1.TXT')
dataB2 = readData_new('MeasuramentsData/Federtyp B_2.TXT')
dataB3 = readData_new('MeasuramentsData/Federtyp B_3.TXT')
dataB4 = readData_new('MeasuramentsData/Federtyp B_4.TXT')

dataB=pd.concat([dataB1,dataB2,dataB3,dataB4])

dataEF1 = readData_new('MeasuramentsData/Federtyp EF_1.TXT')
dataEF2 = readData_new('MeasuramentsData/Federtyp EF_2.TXT')
dataEF3 = readData_new('MeasuramentsData/Federtyp EF_3.TXT')
dataEF4 = readData_new('MeasuramentsData/Federtyp EF_4.TXT')

dataEF= pd.concat([dataEF1,dataEF2,dataEF3,dataEF4])

dataF1 = readData_new('MeasuramentsData/Federtyp F_1.TXT')
dataF2 = readData_new('MeasuramentsData/Federtyp F_2.TXT')
dataF3 = readData_new('MeasuramentsData/Federtyp F_3.TXT')
dataF4 = readData_new('MeasuramentsData/Federtyp F_4.TXT')

dataF= pd.concat([dataF1,dataF2,dataF3,dataF4])

dataM1 = readData_new('MeasuramentsData/Federtyp M_1.TXT')
dataM2 = readData_new('MeasuramentsData/Federtyp M_2.TXT')
dataM3 = readData_new('MeasuramentsData/Federtyp M_3.TXT')
dataM4 = readData_new('MeasuramentsData/Federtyp M_4.TXT')

dataM= pd.concat([dataM1,dataM2,dataM3,dataM4])

data_final=pd.concat([dataB,dataEF,dataF,dataM],keys=['B','EF','F','M'],ignore_index=True, sort=False)
                      
data_final.tail()


,Datum Uhrzeit,Schreiben 0/0 [V],Schreiben 0/1 [V],Schreiben 0/2 [V],Schreiben 0/3 [V],Schreiben 0/4 [V],Schreiben 0/5 [V],Schreiben 0/6 [V],Schreiben 0/7 [V],Schreiben 0/8 [V],...,Schreiben 5/7 [V],Schreiben 5/8 [V],Schreiben 5/9 [V],Schreiben 5/10 [V],Schreiben 5/11 [V],Schreiben 5/12 [V],Schreiben 5/13 [V],Schreiben 5/14 [V],Schreiben 5/15 [V],Schreiben 6/0 [V]
1643,"05.09.2018 11:22:53,00",47.263234,47.263234,8.794629,8.090293,19.324654,19.324654,4.233085,4.338852,875.000000,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
1644,"05.09.2018 11:23:06,00",55.264633,55.264633,15.628406,1.541478,31.590973,31.590973,8.059764,0.658376,870.833333,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
1645,"05.09.2018 11:23:44,00",25.853471,25.853471,2.475847,2.346858,11.519591,11.519591,1.198029,1.180287,879.166667,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
1646,"05.09.2018 11:23:56,00",99.962614,99.962614,6.978670,3.791749,34.240799,34.240799,4.036537,1.681003,904.166667,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
1647,"05.09.2018 11:24:30,00",139.103249,139.103249,27.018073,6.889658,59.029780,59.029780,11.444529,4.064404,866.666667,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,6.0


In [77]:
data_final.shape

(1648, 98)

In [78]:
data_final= data_final.rename(columns={"Schreiben 6/0 [V]": "class"})

In [79]:
X=data_final.drop(['class','Datum Uhrzeit'],axis=1)
y=data_final['class']

In [80]:
X=X.iloc[:,0:48]

In [81]:
y[y!=0]=1

/home/mihir/anaconda3/lib/python3.7/site-packages/ipykernel_launcher.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  """Entry point for launching an IPython kernel.


In [82]:
from sklearn.model_selection import train_test_split
# Splitting train-test set
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.33, random_state=66)

In [83]:

knnlearn = KNeighborsClassifier(n_neighbors=10)
knnlearn.fit(X_train, y_train) 
y_predicted = knnlearn.predict(X_test)

In [84]:
from sklearn.model_selection import cross_val_score
from sklearn.metrics import classification_report, confusion_matrix

In [70]:
print("=== Confusion Matrix KNN Classifier===")
print(confusion_matrix(y_test, y_predicted))
print('\n')


=== Confusion Matrix KNN Classifier===
[[290  27]
 [ 83 144]]




In [71]:
print("=== Classification Report KNN Classifier===")
print(classification_report(y_test, y_predicted))
print('\n')


=== Classification Report KNN Classifier===
              precision    recall  f1-score   support

         0.0       0.78      0.91      0.84       317
         1.0       0.84      0.63      0.72       227

   micro avg       0.80      0.80      0.80       544
   macro avg       0.81      0.77      0.78       544
weighted avg       0.80      0.80      0.79       544





In [72]:
from sklearn.metrics import accuracy_score
print("Accuracy in KNN Classifier", accuracy_score(y_test,y_predicted)*100)

Accuracy in KNN Classifier 79.77941176470588
